 This project contains voting data provided by [YouGov](https://www.voterstudygroup.org/publication/2019-voter-survey-full-data-set) which contains information on peronal characteristics and answers to survey questions. Most of the surveyed people participated in ealrier surveys available on the companies hompage.<br>
This project tough only focuses on the cross-section. Extensions are possible. <br>
Since the survey contains questions and personal characteristics in dimensions greater than 1000 the aim of this project is to twofold: <br>
First to reduce the dimension in order to obtain voter categories determined by a small number of parameters. <br>
Second to check wheter membership to a cretain group can reliable predict voting behavior.

In [41]:
options(messages = -1) # does not include errors and warnings
library(tidyverse)
library(data.table)
# library(glmnet)
library(factoextra)

In [42]:
path_sven <- "~/06_data/VOTER_Survey_Jan217_Release1-csv.csv"
data <- read.csv(file = path_sven, header = TRUE, sep = ",")
head(data)
str(data)

,weight_2016,weight_2017,weight_panel_2018,weight_latino_2018,weight_18_24_2018,weight_overall_2018,weight_2019,weight1_2018,weight1_2019,weight2_2019,⋯,phone_baseline,faminc_baseline,cand_baseline,cdid_baseline,demnom2_baseline,demnom_t_baseline,izip_baseline,lookupzip_baseline,prim12_baseline,statefips_baseline
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>
1,0.3582134,0.4381949,0.5025437,NA,NA,0.3595100,0.3613580,0.2290783,0.2100387,0.6184318,⋯,1,6,258342,03,1,,89052,89052,1,32
2,0.5628674,0.3659623,0.3886910,NA,NA,0.7363869,NA,0.4530128,NA,NA,⋯,1,8,317635,6,3,,85298,85298,2,04
3,0.5521375,0.5498009,0.6841928,NA,NA,0.6316563,0.5946430,0.5321627,0.4652224,0.6123074,⋯,1,5,102825,6,2,Hillery Clinton,54904,54904,1,55
4,0.2075908,NA,NA,NA,NA,NA,0.2738895,NA,NA,0.1869269,⋯,1,9,45897,1,1,,74104,74104,3,40
5,0.3337291,0.3459162,0.3224876,NA,NA,0.4880045,0.5207127,0.3515442,0.3609241,0.3694292,⋯,1,10,330742,21,3,,78148,78148,2,48
6,0.2071856,0.1476409,0.5941112,NA,NA,0.5138021,0.5191906,0.5205669,0.7431791,0.7727754,⋯,3,4,101855,12,1,,94061,94061,1,06


'data.frame':	9548 obs. of  1282 variables:
 $ weight_2016                     : num  0.358 0.563 0.552 0.208 0.334 ...
 $ weight_2017                     : num  0.438 0.366 0.55 NA 0.346 ...
 $ weight_panel_2018               : num  0.503 0.389 0.684 NA 0.322 ...
 $ weight_latino_2018              : num  NA NA NA NA NA NA NA NA NA NA ...
 $ weight_18_24_2018               : num  NA NA NA NA NA NA NA NA NA NA ...
 $ weight_overall_2018             : num  0.36 0.736 0.632 NA 0.488 ...
 $ weight_2019                     : num  0.361 NA 0.595 0.274 0.521 ...
 $ weight1_2018                    : num  0.229 0.453 0.532 NA 0.352 ...
 $ weight1_2019                    : num  0.21 NA 0.465 NA 0.361 ...
 $ weight2_2019                    : num  0.618 NA 0.612 0.187 0.369 ...
 $ weight3_2019                    : num  0.255 NA 0.598 NA 0.476 ...
 $ cassfullcd                      : int  602 405 5506 4001 4828 602 2106 5501 613 5001 ...
 $ vote2020_2019                   : int  2 NA 2 2 1 2 2 NA N

the data set contains 1282 variables where the first 11 columns are population weights and not of interest in further analysis.

# 1. Data Cleaning

Since the dataset contains quite a lot of variables in further analysis we will focus on only a subset for reasons of tractability

In [43]:
# weights not of interest
weight_idx <- grepl(".*weight.*", colnames(data)) 
data <- data[,-weight_idx]
# name of candidate not of interest
name_idx <- grepl(".*Cand.Name.*", colnames(data)) 
data <- data[,-name_idx]
# response to july 2017 survey
survey2017_idx <- grepl(".*_2017", colnames(data)) 
data <- data[,-survey2017_idx]
#response to may 2018 survey
survey2018_idx <- grepl(".*_2018", colnames(data)) 
data <- data[,-survey2018_idx]
# to be continued

In [44]:
head(data)
str(data)

,weight_18_24_2018,weight_overall_2018,weight_2019,weight1_2018,weight1_2019,weight2_2019,weight3_2019,cassfullcd,vote2020_2019,trumpapp_2019,⋯,phone_baseline,faminc_baseline,cand_baseline,cdid_baseline,demnom2_baseline,demnom_t_baseline,izip_baseline,lookupzip_baseline,prim12_baseline,statefips_baseline
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>
1,NA,0.3595100,0.3613580,0.2290783,0.2100387,0.6184318,0.2551877,602,2,4,⋯,1,6,258342,03,1,,89052,89052,1,32
2,NA,0.7363869,NA,0.4530128,NA,NA,NA,405,NA,NA,⋯,1,8,317635,6,3,,85298,85298,2,04
3,NA,0.6316563,0.5946430,0.5321627,0.4652224,0.6123074,0.5980299,5506,2,4,⋯,1,5,102825,6,2,Hillery Clinton,54904,54904,1,55
4,NA,NA,0.2738895,NA,NA,0.1869269,NA,4001,2,4,⋯,1,9,45897,1,1,,74104,74104,3,40
5,NA,0.4880045,0.5207127,0.3515442,0.3609241,0.3694292,0.4756676,4828,1,2,⋯,1,10,330742,21,3,,78148,78148,2,48
6,NA,0.5138021,0.5191906,0.5205669,0.7431791,0.7727754,0.4580058,602,2,4,⋯,3,4,101855,12,1,,94061,94061,1,06


'data.frame':	9548 obs. of  1278 variables:
 $ weight_18_24_2018               : num  NA NA NA NA NA NA NA NA NA NA ...
 $ weight_overall_2018             : num  0.36 0.736 0.632 NA 0.488 ...
 $ weight_2019                     : num  0.361 NA 0.595 0.274 0.521 ...
 $ weight1_2018                    : num  0.229 0.453 0.532 NA 0.352 ...
 $ weight1_2019                    : num  0.21 NA 0.465 NA 0.361 ...
 $ weight2_2019                    : num  0.618 NA 0.612 0.187 0.369 ...
 $ weight3_2019                    : num  0.255 NA 0.598 NA 0.476 ...
 $ cassfullcd                      : int  602 405 5506 4001 4828 602 2106 5501 613 5001 ...
 $ vote2020_2019                   : int  2 NA 2 2 1 2 2 NA NA 2 ...
 $ trumpapp_2019                   : int  4 NA 4 4 2 4 4 NA NA 4 ...
 $ fav_trump_2019                  : int  4 NA 4 4 2 4 4 NA NA 4 ...
 $ fav_obama_2019                  : int  1 NA 1 1 2 1 1 NA NA 98 ...
 $ fav_hrc_2019                    : int  1 NA 1 4 4 1 3 NA NA 1 ...
 $ fav_sande

In [45]:
# some int must be converted to factor

In [46]:
# response variabe of interest
# 1 -> Donald Trump
# 2 -> Dem
# 3 -> no vote
# 4 -> not sure
# 8 -> skipped
str(data$vote2020_2019)
table(data$vote2020_2019)
y_col <- grep("vote2020_2019", colnames(data)) 
y <- data[,y_col]
y <- factor(y)
levels(y) <-  c("Donald_Trump", "Democrat", "No_Vote", "Not_Sure", "Skipped")    
data <- data[,-y_col]

 int [1:9548] 2 NA 2 2 1 2 2 NA NA 2 ...



   1    2    3    4    8 
2630 3266  153  724    6 

# 2. 

In [13]:
set.seed(123)
n <- nrow(data)
p = ncol(data) - 1
ind_train <- sample(x = 1:n, size = ceiling(0.8 * n))
set_train <- data[ind_train,]
ind_test <- setdiff(x = 1:n, ind_train)
set_test <- data[ind_test,]

[1] Democrat     <NA>         Democrat     Democrat     Donald Trump
   [6] Democrat     Democrat     <NA>         <NA>         Democrat    
  [11] <NA>         Democrat     <NA>         Democrat     Democrat    
  [16] Democrat     Democrat     Donald Trump Democrat     Democrat    
  [21] Donald Trump Democrat     Donald Trump Democrat     Donald Trump
  [26] Donald Trump <NA>         <NA>         Donald Trump <NA>        
  [31] Democrat     Donald Trump Donald Trump Donald Trump Democrat    
  [36] Donald Trump Donald Trump Donald Trump Democrat     Democrat    
  [41] Democrat     <NA>         Donald Trump Donald Trump Donald Trump
  [46] Democrat     <NA>         <NA>         <NA>         <NA>        
  [51] Donald Trump <NA>         Democrat     Democrat     <NA>        
  [56] Donald Trump Not Sure     Democrat     Democrat     <NA>        
  [61] Democrat     Democrat     Not Sure     Donald Trump Donald Trump
  [66] Not Sure     Democrat     <NA>         Democrat     <NA>        
  [71] <NA>         Democrat     Democrat     <NA>         Democrat    
  [76] Donald Trump Donald Trump <NA>         <NA>         Democrat    
  [81] Democrat     Donald Trump Democrat     Donald Trump Democrat    
  [86] <NA>         Democrat     Donald Trump Donald Trump Democrat    
  [91] Democrat     Donald Trump Democrat     Donald Trump <NA>        
  [96] Democrat     Not Sure     <NA>         Democrat     Democrat    
 [101] <NA>         Democrat     <NA>         Democrat     Democrat    
 [106] <NA>         Not Sure     <NA>         Democrat     Democrat    
 [111] <NA>         Donald Trump Democrat     <NA>         Democrat    
 [116] Democrat     Donald Trump Not Sure     Donald Trump Not Sure    
 [121] Not Sure     Democrat     <NA>         Democrat     <NA>        
 [126] Democrat     Donald Trump Donald Trump <NA>         Democrat    
 [131] Not Sure     Democrat     <NA>         Donald Trump Democrat    
 [136] Democrat     <NA>         Donald Trump <NA>         <NA>        
 [141] <NA>         Democrat     Donald Trump Donald Trump <NA>        
 [146] <NA>         <NA>         Democrat     Donald Trump Democrat    
 [151] Democrat     <NA>         Donald Trump Democrat     <NA>        
 [156] Democrat     Donald Trump Not Sure     Democrat     Donald Trump
 [161] Democrat     <NA>         Donald Trump <NA>         Donald Trump
 [166] Not Sure     Donald Trump Donald Trump Democrat     Donald Trump
 [171] Donald Trump Not Sure     <NA>         Donald Trump Democrat    
 [176] Donald Trump Democrat     <NA>         Democrat     Donald Trump
 [181] Democrat     <NA>         Donald Trump Not Sure     Donald Trump
 [186] Democrat     Democrat     Democrat     Donald Trump <NA>        
 [191] Democrat     Donald Trump Democrat     Democrat     Donald Trump
 [196] Democrat     Democrat     Democrat     Democrat     Not Sure    
 [201] Donald Trump Democrat     Democrat     Democrat     <NA>        
 [206] Democrat     Democrat     Democrat     Democrat     Donald Trump
 [211] Democrat     Democrat     Donald Trump <NA>         Democrat    
 [216] Donald Trump <NA>         <NA>         Democrat     Democrat    
 [221] Democrat     Donald Trump Democrat     Democrat     Not Sure    
 [226] Not Sure     Democrat     Democrat     Democrat     Not Sure    
 [231] Donald Trump Donald Trump Donald Trump Donald Trump Donald Trump
 [236] Democrat     <NA>         Donald Trump <NA>         Donald Trump
 [241] <NA>         Donald Trump <NA>         Donald Trump <NA>        
 [246] <NA>         Democrat     Donald Trump Democrat     <NA>        
 [251] <NA>         <NA>         Democrat     <NA>         Donald Trump
 [256] Donald Trump <NA>         Not Sure     Donald Trump Donald Trump
 [261] Donald Trump Democrat     Not Sure     <NA>         Democrat    
 [266] Donald Trump Not Sure     Donald Trump Not Sure     Donald Trump
 [271] Donald Trump Donald Trump <NA>         Donald Trump Donald Trump
 [276] Donald Trump Donald Trump Donald Tru

# 3. Neural Net

# 4. LASSO

# 5. Latent Drichilet Allocation

In [5]:
options(messages = 1)